# WeRateDogs Twitter Data Analysis

In [1]:
import pandas as pd
import tweepy
import numpy as np
import json
import os
import requests

## Gather Data

In this section we'll gather the data from multiple sources:
- WeRateDogs Twitter archive
- Tweet image predictions
- Retweet and favorite counts from Twitter API

### WeRateDogs Twitter Archive

The data will be loaded from the CSV file `twitter-archive-enhanced.csv`.

In [2]:
twitter_df = pd.read_csv('twitter-archive-enhanced.csv')
twitter_df.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


### Tweet Image Predictions

The data will be retrieved from the following URL:  
https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv

In [3]:
if 'image_predictions.tsv' in os.listdir():
    print('"image_predictions.tsv" file already exists, retrieval will be skipped.')
else:
    response = requests.get('https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv')
    with open('image_predictions.tsv', 'wb') as file:
        file.write(response.content)

"image_predictions.tsv" file already exists, retrieval will be skipped.


In [4]:
image_pred_df = pd.read_csv('image_predictions.tsv', sep='\t')
image_pred_df.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


### Get Retweet Counts and Like Counts from Tweepy

We'll use Tweepy to get the `retweet_count` and `favorite_count` of every tweet in `twitter_df`. Note that some of the tweets may no longer exists, and for these the `retweet_count` and `favorite_count` will not be available.

In [5]:
if 'tweet_json.txt' in os.listdir():
    print('"tweet_json.txt" file already exists, retrieval will be skipped.')
else:
    tweepy_auth_dir = 'auth/tweepy_auth.json'
    with open(tweepy_auth_dir, 'r') as file:
        tweepy_auth_json = json.load(file)

    key = tweepy_auth_json['key']
    secret = tweepy_auth_json['secret']
    auth = tweepy.OAuthHandler(key, secret)
    api = tweepy.API(auth)
    
    for tweet_id in twitter_df.tweet_id:
        try:
            status = api.get_status(tweet_id, tweet_mode='extended')
            with open('tweet_json.txt', 'a+') as out_file:
                json.dump(status._json, out_file)
                out_file.write('\n')
        except:
            pass

"tweet_json.txt" file already exists, retrieval will be skipped.


In [6]:
tweet_infos = []
with open('tweet_json.txt', 'r') as file:
    for line in file:
        tweet_json = json.loads(line)
        tweet_infos.append({'tweet_id': tweet_json['id_str'],
                           'retweet_count': tweet_json['retweet_count'],
                           'favorite_count': tweet_json['favorite_count']})
tweet_infos_df = pd.DataFrame(tweet_infos)
tweet_infos_df.to_csv('tweet_infos.csv', index=False)
tweet_infos_df.head()

,tweet_id,retweet_count,favorite_count
0,892420643555336193,7417,35140
1,892177421306343426,5517,30411
2,891815181378084864,3640,22869
3,891689557279858688,7584,38400
4,891327558926688256,8166,36651


## Data Assessment and Cleaning

### Data Completeness - `twitter_df`

#### Assessment

In [7]:
twitter_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

As you can see above, the following columns have null values: 
- `in_reply_to_status_id`
- `in_reply_to_user_id`
- `retweeted_status_id`
- `retweeted_status_user_id`
- `retweeted_status_timestamp`
- `expanded_urls`

For the `in_reply...` and `retweeted_...` columns, they make sense to have null values, because not all statuses are replying or retweeting another status. As for `expanded_urls`, having null values in them is not an issue, because we most likely won't use this column. So in conclusion, no cleaning action to be done relating to missing data.

### Data Completeness - `tweet_infos_df`

#### Assessment

In [122]:
tweet_infos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2043 entries, 0 to 2042
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tweet_id        2043 non-null   object
 1   retweet_count   2043 non-null   int64 
 2   favorite_count  2043 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 48.0+ KB


No missing value found, hence no cleaning needed.

### Data Completeness - `image_pred_df`
#### Assessment

In [123]:
image_pred_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


No missing value found, hence no cleaning needed.

### Data Tidiness
#### Assessment

# --------------- DRAFT ---------------

### Change Index to `tweet_id`

In [128]:
twitter_df.tweet_id.unique().shape

(2356,)

In [131]:
twitter_df.shape

(2356, 17)

As seen above, the number of unique values for the tweet_ids are the same as the total number of rows. This means we can replace the tweet_id to be the index.

#### Cleaning

In [134]:
twitter_df.set_index('tweet_id', inplace=True)
twitter_df.head()

,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
tweet_id,,,,,,,,,,,,,,,,
892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,10,Phineas,NaN,NaN,NaN,NaN
892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https:/...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,10,Tilly,NaN,NaN,NaN,NaN
891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181378084864/photo/1,12,10,Archie,NaN,NaN,NaN,NaN
891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557279858688/photo/1,13,10,Darla,NaN,NaN,NaN,NaN
891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https:/...",NaN,NaN,NaN,"https://twitter.com/dog_rates/status/891327558926688256/photo/1,https://twitter.com/dog_rates/status/891327558926688256/photo/1",12,10,Franklin,NaN,NaN,NaN,NaN


### Dog Stage Columns - "None" String Values
#### Assessment
Let's now assess the `doggo`, `floofer`, `pupper`, and `puppo`, which represents the various dog "stages".

In [76]:
print(twitter_df.doggo.unique())
print(twitter_df.floofer.unique())
print(twitter_df.pupper.unique())
print(twitter_df.puppo.unique())

['None' 'doggo']
['None' 'floofer']
['None' 'pupper']
['None' 'puppo']


As you can see, there are string "None" values in those columns, which are invalid and misleading. This can make it hard for programmatic analysis later on.

#### Cleaning
We'll replace the string "None" values to be actual `None` values to ease the analysis.

In [114]:
new_cols = twitter_df[['doggo', 'floofer', 'pupper', 'puppo']].replace('None', None)
twitter_df[['doggo', 'floofer', 'pupper', 'puppo']] = new_cols

#### Re-assessment

It is confirmed below that the "None" string values have been replaced with actual `None` values, shown by the `nan` in the list of unique values.

In [116]:
print(twitter_df.doggo.unique())
print(twitter_df.floofer.unique())
print(twitter_df.pupper.unique())
print(twitter_df.puppo.unique())

[nan 'doggo']
[nan 'floofer']
[nan 'pupper']
[nan 'puppo']


Below we'll reassess the number of null values for these dog stages columns.

In [118]:
twitter_df[['doggo', 'floofer', 'pupper', 'puppo']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   doggo    97 non-null     object
 1   floofer  10 non-null     object
 2   pupper   257 non-null    object
 3   puppo    30 non-null     object
dtypes: object(4)
memory usage: 73.8+ KB


As you can see now apparently there are a lot of null values for the dog stages columns. However, as we'll see in the next section, these columns actually only represents the existence of those dog stages word in the tweet status. This means it makes sense to have a lot of null values, because not all tweets will mention those dog stages words. Hence no cleaning action will be done regarding these missing dog stages column values.

### Dog Stage Columns - Multiple Dog Stages in a Row (while actually there is only one dog)

#### Assessment
Next, let's see whether is it possible for a row to have multiple dog stages.

In [92]:
dog_stages_count = (twitter_df[['doggo', 'floofer', 'pupper', 'puppo']] != 'None').sum(axis=1)
print(f'Possible number of dog stages in a row: {dog_stages_count.unique()}')

Possible number of dog stages in a row: [0 1 2]


There are indeed suprisingly some rows that have 2 dog stages. Since this dog stage information is derived from the corresponding tweet status text, let's now see the corresponding statuses of these rows.

In [121]:
twitter_df.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,10,Phineas,NaN,NaN,NaN,NaN
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https:/...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,10,Tilly,NaN,NaN,NaN,NaN
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181378084864/photo/1,12,10,Archie,NaN,NaN,NaN,NaN
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557279858688/photo/1,13,10,Darla,NaN,NaN,NaN,NaN
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https:/...",NaN,NaN,NaN,"https://twitter.com/dog_rates/status/891327558926688256/photo/1,https://twitter.com/dog_rates/status/891327558926688256/photo/1",12,10,Franklin,NaN,NaN,NaN,NaN


In [120]:
pd.options.display.max_colwidth = 150 # so we can see the whole status text
statuses = twitter_df[dog_stages_count >= 2].text.values
for s in statuses:
    print('- ' + s)

- Here's a puppo participating in the #ScienceMarch. Cleverly disguising her own doggo agenda. 13/10 would keep the planet habitable for https://t.co/cMhq16isel
- At first I thought this was a shy doggo, but it's actually a Rare Canadian Floofer Owl. Amateurs would confuse the two. 11/10 only send dogs https://t.co/TXdT3tmuYk
- This is Dido. She's playing the lead role in "Pupper Stops to Catch Snow Before Resuming Shadow Box with Dried Apple." 13/10 (IG: didodoggo) https://t.co/m7isZrOBX7
- Here we have Burke (pupper) and Dexter (doggo). Pupper wants to be exactly like doggo. Both 12/10 would pet at same time https://t.co/ANBpEYHaho
- Like doggo, like pupper version 2. Both 11/10 https://t.co/9IxWAXFqze
- This is Bones. He's being haunted by another doggo of roughly the same size. 12/10 deep breaths pupper everything's fine https://t.co/55Dqe0SJNj
- This is Pinot. He's a sophisticated doggo. You can tell by the hat. Also pointier than your average pupper. Still 10/10 would pet cautiou

Evaluating the list of twitter statuses above (and also actually opening the pictures of the tweets), most of them have multiple dog stages because the status and the picture itself are involving more than one dogs, which totally makes sense. However, some rows actually only have one dog, but the tweet status mentions multiple dog stages. For these cases, the multiple dog stages will indeed be misleading.

### Data Validity

In [30]:
twitter_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

#### Data Types
Based on the `info()` above, we can see several issues:
- `tweet_id` is of type int
- `timestamp` and `retweeted_status_timestamp` are of type string
- `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id` are of type float

### Column-By-Column Assessment
Now we'll do the data assessment on column-by-column basis, and will assess each column for the following data quality (in sequence):
- Completeness
- Validity
- Accuracy
- Consistency

#### `tweet_id`

In [12]:
twitter_df.tweet_id.describe()

count    2.356000e+03
mean     7.427716e+17
std      6.856705e+16
min      6.660209e+17
25%      6.783989e+17
50%      7.196279e+17
75%      7.993373e+17
max      8.924206e+17
Name: tweet_id, dtype: float64

`tweet_id` looks fine. There seems to be no weird values of the `tweet_id`.

#### `timestamp`

In [28]:
twitter_df.timestamp.str.len().unique()

array([25], dtype=int64)

All timestamps have same length, so this looks good. Visual assessment of the column also does not spot any issue.

#### `source`

In [29]:
twitter_df.source

0       <a href="http://twitter.com/download/iphone" r...
1       <a href="http://twitter.com/download/iphone" r...
2       <a href="http://twitter.com/download/iphone" r...
3       <a href="http://twitter.com/download/iphone" r...
4       <a href="http://twitter.com/download/iphone" r...
                              ...                        
2351    <a href="http://twitter.com/download/iphone" r...
2352    <a href="http://twitter.com/download/iphone" r...
2353    <a href="http://twitter.com/download/iphone" r...
2354    <a href="http://twitter.com/download/iphone" r...
2355    <a href="http://twitter.com/download/iphone" r...
Name: source, Length: 2356, dtype: object

### Data Accuracy

Some issue with `rating_denominator` down there.

In [37]:
twitter_df.rating_denominator.unique()

array([ 10,   0,  15,  70,   7,  11, 150, 170,  20,  50,  90,  80,  40,
       130, 110,  16, 120,   2], dtype=int64)

## Data Assessment Summary
- Data tidiness:
    - dog stage columns have `None` values
   

# Clean Data